# 数字图像处理实验4：二维码识别

成员：19281171王雨潇，19301153潘雪菲

## 导入包

In [8]:
import cv2
import tkinter
import pyzbar.pyzbar as pyzbar

## 设置摄像头

In [9]:
# tkinter 包读取屏幕宽度和高度
screen = tkinter.Tk()
screen_width = screen.winfo_screenwidth()
screen_height = screen.winfo_screenheight()

# 使用笔记本前置摄像头
camera = cv2.VideoCapture(0)

# 根据屏幕大小设置相机的分辨率
camera.set(3, screen_width * 2 / 3) 
camera.set(4, screen_height * 2 / 3)

True

## 定义颜色常量

In [10]:
# OpenCV 的颜色顺序是 (B, G, R)
Blue = (255, 0, 0)
Green = (0, 255, 0)
Red = (0, 0, 255)

## 定义收集链接的容器

In [11]:
# 初始化为空
link_map = set([])

## 定义画面预处理函数

In [12]:
# 求均值白平衡法
def white_balance(img):
    # 读取图像
    b, g, r = cv2.split(img)
    # 求各通道平均值
    b_avg = cv2.mean(b)[0]
    g_avg = cv2.mean(g)[0]
    r_avg = cv2.mean(r)[0]
    # 求各个通道所占增益
    k = (r_avg + g_avg + b_avg) / 3
    kb = k / b_avg
    kg = k / g_avg
    kr = k / r_avg
    # 重新调整权重，生成新图像
    r = cv2.addWeighted(src1=r, alpha=kr, src2=0, beta=0, gamma=0)
    g = cv2.addWeighted(src1=g, alpha=kg, src2=0, beta=0, gamma=0)
    b = cv2.addWeighted(src1=b, alpha=kb, src2=0, beta=0, gamma=0)
    balance_img = cv2.merge([b, g, r])
    return balance_img


# 彩色图像进行自适应直方图均衡化
def histogram_balance(img):
    # RGB 转换到 YCrCb 空间
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
    # 将 YCrCb 图像通道分离
    channels = cv2.split(ycrcb)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe.apply(channels[0], channels[0])
    # 对彩色图像进行直方图均衡化，先将彩色图像分解成若干通道，各通道分别进行直方图均衡化，再合并所有通道
    cv2.merge(channels, ycrcb)
    cv2.cvtColor(ycrcb, cv2.COLOR_YCR_CB2BGR, img)
    return img

# 图像锐化
def unsharpen_mask(img): 
    # 原图的高斯模糊
    blur = cv2.GaussianBlur(img, (5, 5), 1)
    # 结果 = (a * 原图) + (b * 原图的高斯模糊)
    # 需要满足其中a >= 1 && b <= 0 && a + b == 1
    dst = cv2.addWeighted(img, 2, blur, -1, 0)
    return dst
        

## 读取视频并识别二维码

In [13]:
while True:
    # 按帧读取视频
    successed, frame = camera.read()
    # 除错，没读到画面就退出
    if (successed == False):
        print("Error: Can't read camera!\n")
        break

    # 白平衡、直方图均衡化处理、锐化
    frame = white_balance(frame)
    frame = histogram_balance(frame)
    frame = unsharpen_mask(frame)
    # 获取画面中心点
    h1, w1 = frame.shape[0], frame.shape[1] 
    # 用 pyzbar 挑选出图像中所有二维码
    codes = pyzbar.decode(frame)

    # 对画面中的每个二维码都做出处理
    for code in codes:
        # 获取二维码的外接矩形顶点坐标
        (x, y, w, h) = code.rect
        # 获取二维码中心坐标
        cx = int(x + w / 2)
        cy = int(y + h / 2)
        # 标识二维码中心
        cv2.circle(frame, (cx, cy), 2, Red, 4)  
        # 标识二维码外接矩形
        cv2.rectangle(frame, (x, y), (x + w, y + h), Red, 2)  
        # 标识二维码内容
        code_text = code.data.decode('utf-8')
        txt = '(' + code.type + ')  ' + code_text
        cv2.putText(frame, txt, (x - 10, y - 10), cv2.FONT_ITALIC, 0.5, Green, 2)
        # 二维码内容添加到容器中
        link_map.add(code_text)
        # 画面左上角标识二维码中心位置
        cv2.putText(frame, 'Press ESC to exit. Found QRCode: ' + str((cx, cy)), (20, 20), cv2.FONT_ITALIC, 0.5, Blue, 2)

    # 展示画面
    cv2.imshow('Lab4', frame)

    # 按 ESC 键退出
    if cv2.waitKey(1) == 27: 
        break

# 释放摄像头
camera.release()
# 关闭所有图像窗口
cv2.destroyAllWindows()

## 显示所有检测到的二维码

In [14]:
print("INFO: Found {} barcode(s): ".format(len(link_map)))
# 遍历容器
for link in link_map:
    print("{}".format(link))

INFO: Found 1 barcode(s): 
https://sayaka-4987.github.io/
